In [2]:
import qml
import numpy as np
import itertools as it
import matplotlib.pyplot as plt
import numpy as np

In [3]:
c = qml.Compound('../../test/c20.xyz')

In [14]:
def get_combinations_knuth_L(seq):
    i_indices = range(len(seq) - 1, -1, -1)
    k_indices = i_indices[1:]
    seq = sorted(seq)

    while True:
        yield seq
        for k in k_indices:
            if seq[k] < seq[k + 1]:
                break
        else:
            return
        k_val = seq[k]
        for i in i_indices:
            if k_val < seq[i]:
                break
        (seq[k], seq[i]) = (seq[i], seq[k])
        seq[k + 1:] = seq[-1:k:-1]
def get_combinations_wrapper(bncount, ndim):
    vals = [5, 7] * bncount + [6]*(ndim-2*bncount)
    for _ in get_combinations_knuth_L(vals):
        yield _
    return
def get_rep(ncharges):
    return qml.fchl.generate_representation(c.coordinates, ncharges)
def get_BN_distance(a):
    changed = [idx for idx, _ in enumerate(a) if _ != 6]
    dist = np.linalg.norm(c.coordinates[changed[0]] - c.coordinates[changed[1]])
    return dist
def get_distances(setA, setB):
    return qml.fchl.get_global_kernels(np.array(setA), np.array(setB), [2.5,], alchemy="off")[0]

In [27]:
def find_similar_dissimilar(mindist, maxdist):
    # get configurations and representations
    opposite_configurations = []
    other_configurations= []
    for combination in get_combinations_wrapper(1, 20):
        representation = get_rep(combination)
        if get_BN_distance(combination) < maxdist and get_BN_distance(combination) > mindist:
            opposite_configurations.append((combination.copy(), representation))
        else:
            other_configurations.append((combination.copy(), representation))
    print ('opposite: (should be 20) ', len(opposite_configurations))
    print ('other: (should be 360) ', len(other_configurations))
    
    # calculate distances
    ds_opp_opp = get_distances([_[1] for _ in opposite_configurations], [_[1] for _ in opposite_configurations])
    ds_opp_other = get_distances([_[1] for _ in other_configurations], [_[1] for _ in opposite_configurations])
    return ds_opp_opp, ds_opp_other, opposite_configurations, other_configurations
    return np.min(ds_opp_opp), np.max(ds_opp_other)
    
#print(find_similar_dissimilar(0, 1.8))
#print(find_similar_dissimilar(1.8, 2.7))
#print(find_similar_dissimilar(2.7, 3.6))
#print(find_similar_dissimilar(3.6, 4.1))
#print(find_similar_dissimilar(4.1, 10))

In [28]:
ds_opp_opp, ds_opp_other, opposite_configurations, other_configurations = find_similar_dissimilar(4.1, 10)

opposite: (should be 20)  20
other: (should be 360)  360


In [59]:
other = other_configurations[np.argmax(np.max(ds_opp_other, axis=1))][1]
opposite = opposite_configurations[np.argmin(np.min(ds_opp_opp, axis=1))][1]

In [65]:
qml.fchl.get_atomic_kernels(np.array(other), np.array(opposite), [0.5,], alchemy="off")

array([[[0.99834117, 0.94617161, 0.0085465 , 0.94617167, 0.99834117,
         0.98580431, 0.98580434, 0.99834117, 0.99834117, 0.94617147,
         0.99834117, 0.99834117, 0.98580432, 0.98580432, 0.94222593,
         0.98580432, 0.98580431, 0.94222554, 0.00854652, 0.9422255 ,
         0.01308326, 0.01308326, 0.01308326],
        [0.98378839, 0.9410292 , 0.00801339, 0.94102931, 0.98378841,
         0.98378834, 0.98378828, 0.98378819, 0.98378826, 0.94102918,
         0.98378829, 0.9837883 , 0.98378833, 0.98378843, 0.94102967,
         0.98378826, 0.98378829, 0.9410292 , 0.00801342, 0.94102911,
         0.01226717, 0.01226717, 0.01226717],
        [0.98378833, 0.9410291 , 0.00801339, 0.94102921, 0.98378835,
         0.98378828, 0.98378822, 0.98378812, 0.9837882 , 0.94102907,
         0.98378823, 0.98378824, 0.98378827, 0.98378837, 0.94102957,
         0.9837882 , 0.98378823, 0.94102911, 0.00801341, 0.94102901,
         0.01226716, 0.01226716, 0.01226716],
        [0.99834117, 0.94617167, 0

In [55]:
other = other_configurations[np.argmax(np.max(ds_opp_other, axis=1))][0]
opposite = opposite_configurations[np.argmin(np.min(ds_opp_opp, axis=1))][0]

In [57]:
np.max(ds_opp_other), np.min(ds_opp_opp)

(0.9998741946369403, 0.9999999999899682)

In [66]:
c1 = qml.Compound('../../test/c20-opposite.xyz')
c2 = qml.Compound('../../test/c20-other.xyz')

In [67]:
r1 = qml.fchl.generate_representation(c1.coordinates, c1.nuclear_charges)
r2 = qml.fchl.generate_representation(c2.coordinates, c2.nuclear_charges)

In [72]:
qml.fchl.get_atomic_kernels(r1, r2, [0.5,])

array([[[0.9998336 , 0.99869457, 0.99869456, 0.9998336 , 0.99412284,
         0.99999999, 0.99555625, 0.99570452, 0.99320025, 0.99570451,
         0.99555625, 0.99999999, 0.99435597, 0.63519233, 0.99435592,
         0.63523849, 0.99306667, 0.99869434, 0.99869433, 0.99306668,
         3.72524027, 3.72524027, 3.72524027],
        [0.99435583, 0.99474183, 0.99474182, 0.99435583, 0.99993429,
         0.99414946, 0.99317255, 0.99303005, 0.98212486, 0.99303005,
         0.99317252, 0.99414944, 0.9998343 , 0.63330091, 0.9998343 ,
         0.63326577, 0.98225033, 0.9944098 , 0.99440981, 0.98225039,
         3.74534486, 3.74534486, 3.74534486],
        [0.63520563, 0.63554463, 0.6355446 , 0.63520565, 0.63326647,
         0.63516555, 0.63524324, 0.63518991, 0.63325665, 0.63518987,
         0.63524313, 0.6351655 , 0.63323816, 0.99993359, 0.63323815,
         0.20682192, 0.63336708, 0.63603437, 0.63603444, 0.63336717,
         3.73742781, 3.73742781, 3.73742781],
        [0.99435583, 0.99474184, 0

In [69]:
c1.nuclear_charges, c2.nuclear_charges

(array([6, 6, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 6]),
 array([6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 5, 6, 7, 6, 6, 6, 6]))